In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import fiddler as fdl

%matplotlib inline

In [ ]:
# use the nicer plotting styles from seaborn
sns.set()

In [ ]:
%load_ext autoreload
%autoreload 2

# Intro
This notebook assumes you already have the models and data you're interested in using uploaded to Fiddler. Please refer to the previous notebook in this series for more information on uploading to Fiddler. You will also need to have run notebook 1 in order to upload the bikesharing example used in this notebook.

In this notebook we run through a number of other Fiddler functionalities that have been integrated into the Python package. Unlike the previous notebook, there is not as much of a sequential flow to the steps demonstrated here.

## Before you start: set up your API connection

### Launch onebox or authenticate with a remote server
Before you can start working with a Fiddler-integrated Jupyter environment, you should set up access to a running instance of Fiddler.

#### Onebox
In onebox, this means running the `start.sh` script to launch onebox locally.

#### Cloud
For the cloud version of our product, this means looking up your authentication token in the [Fiddler settings dashboard](https://app.fiddler.ai/settings/credentials)

### Create a FiddlerApi object

In order to get your data and models into the Fiddler Engine, you'll need to connect using the API. The `FiddlerApi` object to handles most of the nitty-gritty for you, so all you have to do is specify some details about the Fiddler system you're connecting to.

In [ ]:
client = fdl.FiddlerApi()

## Pulling data from Fiddler

In [ ]:
# let's see which datasets we have on Fiddler
client.list_datasets()

In [ ]:
from sklearn.model_selection import train_test_split

dataset_id = 'bike_share'

# client.delete_dataset(dataset_id)
if dataset_id not in client.list_datasets():
    df = pd.read_csv('/app/fiddler_samples/samples/datasets/bike_share/bike_share_train.csv')
    df_schema = fdl.DatasetInfo.from_dataframe(df, max_inferred_cardinality=1000)
    train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
    upload_result = client.upload_dataset(
        dataset={'train': train, 'test': test}, 
        dataset_id=dataset_id,
        info=df_schema)


In [ ]:
# the info for any dataset can quickly and easily be fetched with the `dataset_info` method
bikeshare_dataset_info = client.get_dataset_info(dataset_id)
bikeshare_dataset_info

In [ ]:
# we can also pull data from the dataset directly into Pandas
bikeshare_dataset = client.get_dataset(dataset_id, max_rows=999_999)
print(f'The bikeshare_dataset object is a {type(bikeshare_dataset)} with keys ({list(bikeshare_dataset.keys())})')

df_train = bikeshare_dataset['train']
df_test = bikeshare_dataset['test']

# demo the data
df_train.sample(3, random_state=0).T

In [ ]:
# for example, let's plot a regression plot of the target (cnt) against the temperature feature (temp)
sns.regplot(df_train['temp'], df_train['cnt'], marker='.', scatter_kws=dict(alpha=0.1))

## Using the Fiddler model-builder feature
If you have data but haven't built a model yet, you can take advantage of the model-builder feature to whip up a model instantly so you can dive right into running explanations.

In [ ]:
bikeshare_dataset_info.get_column_names()

In [ ]:
# fiddler_api.delete_model('bikeshare_forecasting', 'generated_bikeshare_model')

In [ ]:
# NOTE: to avoid training on the whole dataset, we pass `train_splits`
project_id = 'tutorial'
model_id='bikeshare_model'
client.delete_model(project_id, model_id)
features = list(set(bikeshare_dataset_info.get_column_names()) - {'casual', 'registered', 'cnt', 'dteday'})
client.create_model(project_id=project_id, 
                         dataset_id=dataset_id,
                         target='cnt',
                         features=features,
                         model_id=model_id,
                         train_splits=['train'])

In [ ]:
# the new model shows up when we list the models in the bikeshare_forecasting project
client.list_models(project_id=project_id)

## Explanations in Jupyter
We also support basic integration of our explanation and prediction functionality in Jupyter. The `FiddlerApi` object is your friend here.

In [ ]:
# running some predictions on the generated model
client.run_model(project_id=project_id, model_id=model_id, df=df_test.head(10))

In [ ]:
# compare against predictions on our kNN model
client.run_model(project_id='bikeshare_forecasting', model_id='knn_model', df=df_test.head(10))

In [ ]:
# Run explanations on both models
selected_point = df_test.head(1)
ex_generated = client.run_explanation(
    project_id=project_id,
    model_id=model_id, 
    df=selected_point, 
    dataset_id=dataset_id)

ex_knn = client.run_explanation(
    project_id=project_id,
    model_id=model_id, 
    df=selected_point, 
    dataset_id=dataset_id)

In [ ]:
# Create a plot comparing attributions
fig = plt.figure(figsize=(12, 6))
comparison_table = pd.DataFrame({
    'Generated Model': pd.Series(ex_generated.attributions, index=ex_generated.inputs),
    'kNN Model': pd.Series(ex_knn.attributions, index=ex_knn.inputs)
})
comparison_table = comparison_table.loc[comparison_table['kNN Model'].abs().sort_values(ascending=False).index, :]

melted_table = (comparison_table
                .reset_index()
                .rename(columns={'index': 'Feature'})
                .melt(id_vars='Feature', 
                      var_name='Model', 
                      value_name='Attribution'))
sns.barplot(x='Attribution', y='Feature', hue='Model', data=melted_table)

plt.title('Top SHAP attributions on first row of bikeshare for generated model')

# Conclusion
As we have seen in this notebook, once data and models have been deployed to Fiddler, it becomes very easy to share the data, automatically train a model on Fiddler, and run explanations all without leaving